In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import calitp_data_analysis.magics

import geopandas as gpd
import pandas as pd

from great_tables import GT

import _report_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS

SHARED_GCS = GTFS_DATA_DICT.gcs_paths.SHARED_GCS

In [ ]:
# Comment out and leave this cell right below pandas
# district = "SD 01"

In [ ]:
legislative_crosswalk = pd.read_parquet(
    f"{SHARED_GCS}crosswalk_transit_operators_legislative_districts.parquet",
    filters = [[("legislative_district", "==", district)]]
)

def readable_district_name(word: str) -> str:
    if "SD" in word:
        return word.replace("SD", "Senate District")
    elif "AD" in word:
        return word.replace("AD", "Assembly District")

district_full_name = readable_district_name(district)

In [ ]:
%%capture_parameters
district, district_full_name

In [ ]:
# Read in all datasets here with GTFS_DATA_DICT 
# Reran merge_operator_data.py to test this
OPERATOR_FILE = GTFS_DATA_DICT.digest_tables.operator_profiles
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map
operator_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_FILE}.parquet",
).merge(
    legislative_crosswalk,
    on = "name",
    how = "inner"
)

# using name instead of schedule_gtfs_dataset_key allows us to get 
# the last ones for LA Metro without keeping extraneous rows for LA Metro when keys changed
operator_df = operator_df.sort_values(
    ["service_date", "name"], 
    ascending=[False, True]
).drop_duplicates(
    subset=["name"]
).reset_index(drop=True)

# {district}

These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

In [ ]:
district_summary = _report_utils.district_stats(
    operator_df, 
    "legislative_district"
)

In [ ]:
summary_table1 = (GT(
    district_summary.drop(
        columns = ["arrivals_per_stop", "trips_per_operator"]
    ).pipe(
        _report_utils.transpose_summary_stats, 
        district_col = "legislative_district"
    )
)
 .fmt_integer(columns="value")
 .cols_label(index="")
 .tab_header(title = f"{district_full_name} GTFS summary stats")
)

summary_table2 = (GT(
    district_summary[
        ["legislative_district", "arrivals_per_stop", "trips_per_operator"]
    ].pipe(
        _report_utils.transpose_summary_stats,
        district_col = "legislative_district"
    )
)
 .fmt_number("value", decimals=1)
 .cols_label(index="")
)

In [ ]:
display(summary_table1)
display(summary_table2)

## GTFS Stats by Operator

In [ ]:
operators_in_district = operator_df.schedule_gtfs_dataset_key.unique()

operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    filters = [["schedule_gtfs_dataset_key", "in", operators_in_district]],
    columns = ["name", "service_date", 
               "route_combined_name", "geometry"]
).sort_values(
    ["service_date", "name", "route_combined_name"], 
    ascending=[False, True, True]
).drop_duplicates(
    subset = ["name", "route_combined_name"]
).drop(
    columns = ["service_date", "route_combined_name"]
    # drop route because after the dissolve, all operator routes are combined
    # so route would hold only the first row's value
).dissolve(by = "name").reset_index().pipe(_report_utils.replace_column_names)

operator_route_gdf["Transit Operator"] = operator_route_gdf["Transit Operator"].str.replace(" Schedule", "")

In [ ]:
operator_route_gdf.explore(
    "Transit Operator", 
    tiles = "CartoDB Positron",
    categorical=True,
    legend=True,
    legend_kwds = {
        "width": 200
    }
)

In [ ]:
shared_cols = ["organization_name", "name"]
exclude_cols = ["schedule_gtfs_dataset_key", "legislative_district",
                "organization_source_record_id", "service_date", 
                "primary_uza"]

gtfs_service_cols = [
    c for c in operator_df.columns 
    if "operator_" in c]

In [ ]:
gtfs_table_df = operator_df[
    shared_cols + gtfs_service_cols
].pipe(_report_utils.replace_column_names)

In [ ]:
string_cols = gtfs_table_df.select_dtypes(include="object").columns.tolist()

gtfs_table = (
    GT(gtfs_table_df.sort_values("# Trips", ascending=False))
    .fmt_integer(
        columns = [
            c for c in gtfs_table_df.columns if c not in 
         ["Operator Service Miles", "Avg Arrivals per Stop"]
            and (c not in string_cols)
        ])
    .fmt_number(
        columns = ["Operator Service Miles", "Avg Arrivals per Stop"], 
        decimals = 1)
    .data_color(
        columns=["# Trips", "Avg Arrivals per Stop"],
        palette=["white", "green"],
        na_color="lightgray")
    .tab_header(
         title = f"{district_full_name}",
         subtitle = "Daily GTFS schedule statistics by operator")
    .cols_align(columns = [c for c in gtfs_table_df.columns if c not in ["Organization", "Transit Operator"]],
        align="center")
    )

gtfs_table = _report_utils.great_table_formatting(gtfs_table)
gtfs_table